In [3]:
#!pip install -U ollama

# STRUCTURED OUTPUT

Questo notebook presenta alcuni test di utilizzo di structured output con Ollama e LLama3.2

Prima di verificare se e come è possibile utilizzare questa funzionalità con Langchain, inizio utilizzando direttamente la libreria di Ollama (che dalla versione rilasciata a dicembre 2024 supporta structured output).

Da : https://pavelbazin.com/post/the-essential-guide-to-large-language-models-structured-output-and-function-calling/#how-structured-output-works

Structured output is a model’s capability to output JSON, acquired during fine-tuning. 

Come primo step provo a verificare come sia sempre stato possibile produrre output in formato JSON utilizzando le opportune istruzioni da Promt e come tale modalità possa presentare problemi dovuti al fatto che non è possibile "controllare" il formato dell'output prodotto

In [4]:
from ollama import chat
import os

Definisco una funzione che, dati:
- un promt
- un messaggio di testo fornito dall'utente
- un modello (nel nostro caso llama3.2

restituisce un output in formato JSON

In [5]:
def eval(prompt: str, message: str, model: str = "llama3.2:3b-instruct-fp16"):
    
    messages = [
        {"role": "system", "content": prompt},
        {"role": "user", "content": message},
    ]

    response = chat(
        messages=messages,
        model=model,
    )
    return response

In [6]:
prompt = """
You are a data parsing assistant. 
User provides a list of groceries. 
Your goal is to output it as JSON.
"""
message = "I'd like to buy some bread, pack of eggs, few apples, and a bottle of milk."

res = eval(prompt=prompt, message=message)

json_data = res['message']['content']

print(json_data)

Here is the grocery list in JSON format:

```
{
  "groceries": [
    {
      "item": "bread",
      "quantity": null
    },
    {
      "item": "eggs",
      "quantity": null
    },
    {
      "item": "apples",
      "quantity": "few"
    },
    {
      "item": "milk",
      "quantity": null
    }
  ]
}
```


L'output prodotto non è JSON ma è una stringa contenente JSON. POtrebbe essere possibile istruire il modello attraverso il prompt per restituire solo JSON ma non sarebbbe sicuramente una soluzione sicura e affidabile al 100%

Questo perchè non è stato utilizzato lo "structured output". <br>
A tal fine ridefinisco la funzione eval() per utilizzarlo, impostando il formato di output a JSON.

In [7]:
def eval(prompt: str, message: str, model: str = "llama3.2:3b-instruct-fp16"):
    
    messages = [
        {"role": "system", "content": prompt},
        {"role": "user", "content": message},
    ]

    response = chat(
        messages=messages,
        model=model,
        # Enable structured ouput capability
        format="json",
    )
    return response

In [8]:
prompt = """
You are a data parsing assistant. 
User provides a list of groceries. 
Your goal is to output it as JSON.
"""
message = "I'd like to buy some bread, pack of eggs, few apples, and a bottle of milk."

res = eval(prompt=prompt, message=message)

json_data = res['message']['content']

print(json_data)

{"groceries": ["bread", "eggs", "apples", "milk"]}


In questo caso abbiamo del JSON valido che è possibile trattare ad esempio per convertirlo in HTML per essere utilizzato da altre funzioni "standard"

In [9]:
import json

def render(data: str) -> str:
    data_dict = json.loads(data)

    return f"""
    <ul>
        {"\n\t".join([ f"<li>{x}</li>" for x in data_dict["groceries"]])}
    </ul>
    """

In [10]:
print(render(json_data))


    <ul>
        <li>bread</li>
	<li>eggs</li>
	<li>apples</li>
	<li>milk</li>
    </ul>
    


In questo caso però il problema è dato dal fatto che non è stato definito uno schema di output per cui il formato JSON di risposta non è stabile tra le diverse invocazioni.<br>

In [11]:
message = "12 eggs, 2 bottles of milk, 6 sparkling waters"
res = eval(prompt=prompt, message=message)

json_data = res['message']['content']
print(json_data)

{"groceries": [
    {"item": "eggs", "quantity": 12},
    {"item": "milk", "quantity": 2},
    {"item": "sparkling water", "quantity": 6}
]}


Per definire un formato di output stabile  è necessario poter definire uno <b>schema</b> di output per il JSON restituito dal modello.

Attraverso structured output è possibile anche definire uno schema di output

In [12]:
prompt = """
You are data parsing assistant. 
User provides a list of groceries. 
Use the following JSON schema to generate your response:

{{
    "groceries": [
        { "name": ITEM_NAME, "quantity": ITEM_QUANTITY }
    ]
}}

Name is any string, quantity is a numerical value.
"""

inputs = [
    "I'd like to buy some bread, pack of eggs, few apples, and a bottle of milk.",
    "12 eggs, 2 bottles of milk, 6 sparkling waters.",
]

for message in inputs:
    res = eval(prompt=prompt, message=message)
    json_data = res['message']['content']
    print(json_data)

{
  "groceries": [
    { 
      "name": "Bread", 
      "quantity": 1
    },
    { 
      "name": "Eggs", 
      "quantity": 1
    },
    { 
      "name": "Apples", 
      "quantity": 2
    },
    { 
      "name": "Milk", 
      "quantity": 1
    }
  ]
}
{
    "groceries": [
        {"name": "eggs", "quantity": 12},
        {"name": "milk", "quantity": 2},
        {"name": "sparkling water", "quantity": 6}
    ]
}


Un altro modo per gestire l'ouput di un llm è quello di definire uno schema e definire un metodo per serializzare il JSON in un oggetto definito in modo che possa essere successivamente utilizzato da altri moduli

In [13]:
from typing import TypeVar, List, Any, Self, Generic, Callable, Optional
from dataclasses import dataclass, field

# Define generic type variable
T = TypeVar("T")


# Immutable grocery item container
@dataclass(frozen=True)
class Item:
    name: str
    quantity: int


# Immutable groceries container
@dataclass(frozen=True)
class Groceries:
    groceries: List[Item]

    @staticmethod
    def serialize(data: Any) -> Self:
        """JSON serialization function."""
        json_data = json.loads(data)
        items = [Item(**item) for item in json_data["groceries"]]

        return Groceries(groceries=items)


# Edited `eval` function to handle types and serialization
def eval(prompt: str, 
         message: str, 
         schema: Generic[T],
         serializer: Callable = None,
         model: str = "llama3.2:3b-instruct-fp16")-> Optional[T]:
    
    messages = [
        {"role": "system", "content": prompt},
        {"role": "user", "content": message},
    ]

    response = chat(
        messages=messages,
        model=model,
        # Enable structured ouput capability
        format="json",
    )
    try:
        data = response['message']['content']
        json_data = json.loads(data)

        if serializer is not None:
            return serializer(data)
        else:
            return schema(**json_data)
    except TypeError as type_error:
        # Happens when dictionary data shape doesn't match provided schema layout.
        return None
    except json.JSONDecodeError as json_error:
        # Happens when LLM outputs incorrect JSON, or ``json`` module fails
        # to parse it for some other reason.
        return None

In [14]:
res = eval(
    prompt=prompt,
    message="I'd like to buy some bread, pack of eggs, few apples, and a bottle of milk.",
    schema=Groceries,
    serializer=Groceries.serialize,
)

# Pretty print it
print(res)


Groceries(groceries=[Item(name='bread', quantity=1), Item(name='eggs', quantity=1), Item(name='apples', quantity=2), Item(name='milk', quantity=1)])


Altra alternativa è quella di utilizzare la funzionalità di structured output resa disponibile da ollama a dicembre 2024.<br>
Grazie a questa funzionalità è possibile definire uno schema utilizzando Pydantic e richiamare il modello llama3 passando lo schema creato da Pydantic.

TODO : Da approfondire come definire un modello pydantic con un oggetto che comprende una lista di altri oggetti

In [15]:
from pydantic import BaseModel

class Item (BaseModel):
    name: str
    quantity: int

class Groceries(BaseModel):
    groceries: List[Item]

print(Groceries.model_json_schema())

{'$defs': {'Item': {'properties': {'name': {'title': 'Name', 'type': 'string'}, 'quantity': {'title': 'Quantity', 'type': 'integer'}}, 'required': ['name', 'quantity'], 'title': 'Item', 'type': 'object'}}, 'properties': {'groceries': {'items': {'$ref': '#/$defs/Item'}, 'title': 'Groceries', 'type': 'array'}}, 'required': ['groceries'], 'title': 'Groceries', 'type': 'object'}


In [17]:
prompt = """
You are a data parsing assistant. 
User provides a list of groceries. 
Your goal is to output it as JSON.
"""
message = "I'd like to buy some bread, pack of eggs, few apples, and a bottle of milk."

messages = [
    {"role": "system", "content": prompt},
    {"role": "user", "content": message},
]

response = chat(
    messages=messages,
    model= "llama3.2:3b-instruct-fp16",
    # Enable structured ouput capability
    format=Groceries.model_json_schema(),
)

groceries = Groceries.model_validate_json(response.message.content)
print(groceries)

groceries=[Item(name='Bread', quantity=1), Item(name='Eggs', quantity=1), Item(name='Apples', quantity=2), Item(name='Milk', quantity=1)]


Function calling is a type of structured output capability of a large language model.”

LLMs don’t call any functions themselves; they suggest which function you should call from pre-defined functions which you provide to the LLM in a prompt.

Therefore, function calling is nothing but structured output, or a special case of structured output. 

Function calling capability is achieved via fine-tuning of a model

it is just JSON formatted output which contains the name of a function to call and parameters for it

In [45]:
def add_two_numbers(a: int, b: int) -> int:
  """
  Add two numbers

  Args:
    a (int): The first number
    b (int): The second number

  Returns:
    int: The sum of the two numbers
  """
  return a + b


def subtract_two_numbers(a: int, b: int) -> int:
  """
  Subtract two numbers
  """
  return a - b

messages = [{'role': 'user',
             'content': 'What is three plus one?'}]

print('Prompt:', messages[0]['content'])

available_functions = {
  'add_two_numbers': add_two_numbers,
  'subtract_two_numbers': subtract_two_numbers,
}

response: ChatResponse = chat(
  model= "llama3.2:3b-instruct-fp16",
  messages=messages,
  tools=[add_two_numbers, subtract_two_numbers],
)

if response.message.tool_calls:
  
  # There may be multiple tool calls in the response
  for tool in response.message.tool_calls:
    
    # Ensure the function is available, and then call it
    if function_to_call := available_functions.get(tool.function.name):
      print('Calling function:', tool.function.name)
      print('Arguments:', tool.function.arguments)
      output = function_to_call(**tool.function.arguments)
      print('Function output:', output)
    else:
      print('Function', tool.function.name, 'not found')

# Only needed to chat with the model using the tool call results
if response.message.tool_calls:

  # Add the function response to messages for the model to use
  messages.append(response.message)
  messages.append({'role': 'tool', 'content': str(output), 'name': tool.function.name})

  # Get final response from model with function outputs
  final_response = chat(
      model= "llama3.2:3b-instruct-fp16",
      messages=messages)
  print('Final response:', final_response.message.content)

else:
  print('No tool calls returned from model')

Prompt: What is three plus one?
Calling function: add_two_numbers
Arguments: {'a': 3, 'b': 1}
Function output: 4
Final response: The answer to the equation 3 + 1 is indeed 4.
